In [1]:
import pandas as pd

In [2]:
pd.options.display.max_colwidth=300

In [3]:
surveyDF=pd.read_csv("survey_results_public.csv")
surveyDF

FileNotFoundError: [Errno 2] No such file or directory: 'survey_results_public.csv'

In [ ]:
surveyDF.drop(['ResponseId','LearnCode','LearnCodeOnline','LearnCodeCoursesCert','OrgSize','VersionControlSystem','SurveyEase', 
               'SurveyLength',
               'TBranch','ICorPM','Knowledge_1',
                       'Knowledge_1','Knowledge_2','Knowledge_3','Knowledge_4','Knowledge_5','Knowledge_6','Knowledge_7',
                       'CodingActivities','YearsCode','PurchaseInfluence','BuyNewTool',
                       'VersionControlSystem','VCInteraction','VCHostingPersonal use','VCHostingProfessional use'
                       ,'OfficeStackAsyncWantToWorkWith','OfficeStackSyncHaveWorkedWith','Blockchain','SOAccount','SOPartFreq',
                       'Frequency_1','Frequency_2','Frequency_3','TimeSearching','TimeAnswering','TrueFalse_1','TrueFalse_2',
                       'TrueFalse_3','ConvertedCompYearly','Accessibility','NEWSOSites','SOVisitFreq','SOComm'
              ,'Onboarding','OfficeStackSyncWantToWorkWith','LanguageWantToWorkWith','WebframeWantToWorkWith',
              'DatabaseWantToWorkWith','NEWCollabToolsWantToWorkWith','PlatformWantToWorkWith','MiscTechWantToWorkWith',
              'ToolsTechWantToWorkWith'],
              axis=1,inplace=True)
surveyDF.head(10)

In [ ]:
surveyDF.dropna(how=("any"),inplace=True)
surveyDF

In [ ]:
newDF=surveyDF.reset_index(drop=True)
newDF

In [ ]:
newDF.info()

In [ ]:
newDF["WorkExp"].astype("float")

In [ ]:
newDF["YearsCodePro"]=newDF["YearsCodePro"].str.replace("Less than 1 year","1")
newDF

In [ ]:
newDF["YearsCodePro"].astype("float")

In [ ]:
newDF.to_csv("clea")